In [0]:
%sql

CREATE TABLE IF NOT EXISTS dim_user (
  login STRING, -- natural key
  premium_user BOOLEAN, -- SCD Type 2
  address STRING, -- SCD Type 2
  phone STRING, -- SCD Type 2, may be NULL
  name STRING, -- SCD Type 1
  surname STRING, -- SCD Type 1
  year_of_birth INT -- SCD Type 1, may be NULL
) STORED AS PARQUET;

In [0]:
%sql
CREATE TABLE  dim_user_production3(
  dim_user_id INT,
  login STRING,
  premium_user BOOLEAN,
  address STRING,
  phone STRING,
  name STRING,
  surname STRING,
  year_of_birth INT,
  scd_version INT,
  scd_start_date STRING,
  scd_end_date STRING,
  scd_active BOOLEAN
) STORED AS PARQUET;

In [0]:
%sql
CREATE TABLE dim_user_new1
STORED AS PARQUET
AS SELECT
  dim_user_id,
  login,
  premium_user,
  address,
  phone,
  name,
  surname,
  year_of_birth,
  scd_version,
  CAST(scd_start_date AS STRING) AS scd_start_date,
  CAST(scd_end_date AS STRING) AS scd_end_date,
  scd_active
FROM dim_user_production3
LIMIT 0;

In [0]:
%sql
INSERT INTO dim_user_production3
VALUES
  (1, 'john123', true, '123 Main St', '123-456-7890', 'John', 'Doe', 1985, 1, '2022-01-01 00:00:00', '9999-12-31 23:59:59', true),
  (2, 'jane456', false, '456 Elm St', NULL, 'Jane', 'Smith', 1990, 1, '2022-01-01 00:00:00', '9999-12-31 23:59:59', true),
  (3, 'alex789', true, '789 Oak St', '987-654-3210', 'Alex', 'Johnson', 1995, 1, '2022-01-01 00:00:00', '9999-12-31 23:59:59', true);
  select *
from dim_user_production3;

dim_user_id,login,premium_user,address,phone,name,surname,year_of_birth,scd_version,scd_start_date,scd_end_date,scd_active
3,alex789,true,789 Oak St,987-654-3210,Alex,Johnson,1995,1,2022-01-01 00:00:00,9999-12-31 23:59:59,true
1,john123,true,123 Main St,123-456-7890,John,Doe,1985,1,2022-01-01 00:00:00,9999-12-31 23:59:59,true
2,jane456,false,456 Elm St,null,Jane,Smith,1990,1,2022-01-01 00:00:00,9999-12-31 23:59:59,true


In [0]:
%sql
CREATE TABLE IF NOT EXISTS dim_user_staging2 (
  login STRING, -- natural key
  premium_user BOOLEAN, -- SCD Type 2
  address STRING, -- SCD Type 2
  phone STRING, -- SCD Type 2, may be NULL
  name STRING, -- SCD Type 1
  surname STRING, -- SCD Type 1
  year_of_birth INT -- SCD Type 1, may be NULL
) STORED AS PARQUET;

In [0]:
%sql
INSERT INTO dim_user_staging2 (login, premium_user, address, phone, name, surname, year_of_birth)
VALUES ('john123', true, '123 Main St', '555-1234', 'John', 'Doe', 1985),
       ('jane456', false, '456 Elm St', NULL, 'Jane', 'Smith', 1990),
       ('bob789', true, '789 Oak St', '555-9876', 'Bob', 'Johnson', NULL);

select *
from dim_user_staging2;

login,premium_user,address,phone,name,surname,year_of_birth
john123,true,123 Main St,555-1234,John,Doe,1985
bob789,true,789 Oak St,555-9876,Bob,Johnson,null
jane456,false,456 Elm St,null,Jane,Smith,1990


In [0]:
%sql
INSERT INTO dim_user_new1
SELECT p.*
FROM dim_user_production3 p
LEFT JOIN dim_user_staging2 s
ON p.login = s.login
WHERE s.login IS NULL;

 select *
from dim_user_new1;

dim_user_id,login,premium_user,address,phone,name,surname,year_of_birth,scd_version,scd_start_date,scd_end_date,scd_active
3,alex789,true,789 Oak St,987-654-3210,Alex,Johnson,1995,1,2022-01-01 00:00:00,9999-12-31 23:59:59,true


In [0]:
%sql
INSERT INTO TABLE dim_user_new1
SELECT p .dim_user_id,
  p.login,
  p.premium_user,
  p.address,
  p.phone,
  s.name,
  s.surname,
  s.year_of_birth,
  p.scd_version,
  p.scd_start_date,
  p.scd_end_date,
  p.scd_active
FROM dim_user_production3 p
JOIN dim_user_staging2 s
ON p.login = s.login
AND p.scd_active = false;

select *
from dim_user_new1;

dim_user_id,login,premium_user,address,phone,name,surname,year_of_birth,scd_version,scd_start_date,scd_end_date,scd_active
3,alex789,true,789 Oak St,987-654-3210,Alex,Johnson,1995,1,2022-01-01 00:00:00,9999-12-31 23:59:59,true


In [0]:
%sql
INSERT INTO TABLE dim_user_new1
SELECT p.dim_user_id,
  p.login,
  p.premium_user,
  p.address,
  p.phone,
  s.name,
  s.surname,
  s.year_of_birth,
  p.scd_version,
  p.scd_start_date,
  p.scd_end_date,
  p.scd_active
FROM dim_user_production3 p
JOIN dim_user_staging2 s
ON p.login = s.login
AND p.scd_active = true
WHERE p.premium_user = s.premium_user
AND p.address = s.address
AND COALESCE(p.phone, '') = COALESCE(s.phone, ''); --The phone numbers are compared using COALESCE to handle cases where one or both phone numbers are NULL.

select *
from dim_user_new1;

dim_user_id,login,premium_user,address,phone,name,surname,year_of_birth,scd_version,scd_start_date,scd_end_date,scd_active
3,alex789,true,789 Oak St,987-654-3210,Alex,Johnson,1995,1,2022-01-01 00:00:00,9999-12-31 23:59:59,true
2,jane456,false,456 Elm St,null,Jane,Smith,1990,1,2022-01-01 00:00:00,9999-12-31 23:59:59,true


In [0]:
%sql
TRUNCATE TABLE dim_user_new1;

In [0]:
%sql
INSERT INTO TABLE dim_user_new1
SELECT p.dim_user_id,
  p.login,
  p.premium_user,
  p.address,
  p.phone,
  s.name,
  s.surname,
  s.year_of_birth,
  p.scd_version,
  p.scd_start_date,
  '2016-10-01 00:00:00', -- current timestamp for scd_end_date
  false -- false for scd_active
FROM dim_user_production3 p
JOIN dim_user_staging2 s
ON p.login = s.login
AND p.scd_active = true
WHERE p.premium_user != s.premium_user
OR p.address != s.address
OR COALESCE(p.phone, '') != COALESCE(s.phone, ''); 


select *
from dim_user_new1;

dim_user_id,login,premium_user,address,phone,name,surname,year_of_birth,scd_version,scd_start_date,scd_end_date,scd_active
1,john123,true,123 Main St,123-456-7890,John,Doe,1985,1,2022-01-01 00:00:00,2016-10-01 00:00:00,false


In [0]:
%sql
INSERT INTO TABLE dim_user_new1
SELECT n.dim_user_id,
  n.login,
  n.premium_user,
  n.address,
  n.phone,
  n.name,
  n.surname,
  n.year_of_birth,
  n.scd_version,
  '2016-10-01 00:00:00', -- current timestamp for scd_start_date
  '9999-12-31 23:59:59', -- default timestamp for scd_end_date
  true -- true for scd_active
FROM (
  SELECT row_number() OVER (ORDER BY p.login) AS dim_user_id,
    p.login,
    s.premium_user,
    s.address,
    s.phone,
    s.name,
    s.surname,
    s.year_of_birth,
    p.scd_version + 1 AS scd_version
  FROM dim_user_production3 p
  JOIN dim_user_staging2 s
  ON p.login = s.login
  AND p.scd_active = true
  WHERE p.premium_user != s.premium_user
  OR p.address != s.address
  OR COALESCE(p.phone, '') != COALESCE(s.phone, '')
) n,
(
  SELECT MAX(dim_user_id) AS max_id
  FROM dim_user_new1
) m;


select *
from dim_user_new1;

dim_user_id,login,premium_user,address,phone,name,surname,year_of_birth,scd_version,scd_start_date,scd_end_date,scd_active
1,john123,true,123 Main St,123-456-7890,John,Doe,1985,1,2022-01-01 00:00:00,2016-10-01 00:00:00,false
1,john123,true,123 Main St,555-1234,John,Doe,1985,2,2016-10-01 00:00:00,9999-12-31 23:59:59,true


In [0]:
%sql

  INSERT INTO TABLE dim_user_new1
SELECT n.id + COALESCE(m.max_id, 0), -- new id for dim_user_id
  n.login,
  n.premium_user,
  n.address,
  n.phone,
  n.name,
  n.surname,
  n.year_of_birth,
  1, -- 1 for scd_version
  '2016-10-01 00:00:00', -- current timestamp for scd_start_date
  '9999-12-31 23:59:59', -- default timestamp for scd_end_date
  true -- true for scd_active
FROM (
  SELECT row_number() OVER (ORDER BY s.login) AS id,
    s.login,
    s.premium_user,
    s.address,
    s.phone,
    s.name,
    s.surname,
    s.year_of_birth
  FROM dim_user_staging2 s
  LEFT JOIN dim_user_production3 p
  ON s.login = p.login
  WHERE p.login IS NULL
) n,
(
  SELECT MAX(dim_user_id) AS max_id
  FROM dim_user_new1
) m;


select *
from dim_user_new1;

dim_user_id,login,premium_user,address,phone,name,surname,year_of_birth,scd_version,scd_start_date,scd_end_date,scd_active
1,john123,true,123 Main St,123-456-7890,John,Doe,1985,1,2022-01-01 00:00:00,2016-10-01 00:00:00,false
1,john123,true,123 Main St,555-1234,John,Doe,1985,2,2016-10-01 00:00:00,9999-12-31 23:59:59,true
2,bob789,true,789 Oak St,555-9876,Bob,Johnson,null,1,2016-10-01 00:00:00,9999-12-31 23:59:59,true


In [0]:
%sql
INSERT OVERWRITE TABLE dim_user_production3
SELECT *
FROM dim_user_new1;

select *
from dim_user_new1;

dim_user_id,login,premium_user,address,phone,name,surname,year_of_birth,scd_version,scd_start_date,scd_end_date,scd_active
1,john123,true,123 Main St,123-456-7890,John,Doe,1985,1,2022-01-01 00:00:00,2016-10-01 00:00:00,false
1,john123,true,123 Main St,555-1234,John,Doe,1985,2,2016-10-01 00:00:00,9999-12-31 23:59:59,true
2,bob789,true,789 Oak St,555-9876,Bob,Johnson,null,1,2016-10-01 00:00:00,9999-12-31 23:59:59,true
